# FRA measurements with 3rd party devices

If sources or sinks have analog interfaces for control, it is possible to use them for impedance measurement with Thales and the [FRA probe](https://www.zahner.de/products-details/probes/fra-probe).

The manual for the FRA probe explains the use case in detail: [FRA Manual](https://doc.zahner.de/hardware/fra_probe.pdf)

In [6]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import time

# Connect Python to the already launched Thales-Software

In [ ]:

if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "ScriptRemote")
    
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()

# Configuration of the FRA operation

The [FRA Manual](https://doc.zahner.de/hardware/fra_probe.pdf) describes the application in detail, in the following lines only the parameterization relevant for Python is discussed.

The FRA mode must be switched off so that the analog interface to the external device can be initialized.

For the device to work, the gains of voltage and current must be set on the interface. These describe how much signal voltage on the analog interface corresponds to what output current or voltage.
Also minimum and maximum voltage and current of the device must be set.

In the example, the FRA interface is initialized for use with a [DELTA SM3300 SM 18-220](https://www.delta-elektronika.nl/en/products/dc-power-supplies-3300w-sm3300-series.html) power supply/source, which has the analog extension. However, this source is only limited suitable for FRA operation.

This source has **18 V** output voltage and **220 A** maximum current. The analog voltage on the interface is **0 V to 5 V**.

This results in the settings of the gains in the following lines of code. It may be, as in this example, that some gains must have a negative sign. This must be tested with the respective device.

The FRA probe has a maximum +-10 V target signal and one input each for +-10 V current and voltage signal.

The source must be controlled separately so that the analog interface is used, the source is switched on and off or for potentiostatic and galvanostatic. The Thales software only provides the analog signals, everything else must be done via the device interface.



In [ ]:
    zahnerZennium.disableFraMode()
    
    zahnerZennium.setFraVoltageMinimum(0)
    zahnerZennium.setFraVoltageMaximum(18)
    zahnerZennium.setFraCurrentMinimum(0)
    zahnerZennium.setFraCurrentMaximum(220)
    
    zahnerZennium.setFraVoltageInputGain(18.0/5.0)
    zahnerZennium.setFraVoltageOutputGain(18.0/5.0)
    zahnerZennium.setFraCurrentInputGain(-220.0/5.0)
    zahnerZennium.setFraCurrentOutputGain(-220.0/5.0)
    
    zahnerZennium.setFraPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)

After parameterization the FRA mode can be switched on.

In [ ]:
    zahnerZennium.enableFraMode()
    zahnerZennium.setCurrent(0)

At this point, the external device which receives the target signals via FRA must be switched on and configured accordingly.

The output of the device must **not be switched on** beforehand, as the analog interface has not yet been initialized.

Simple DC measurements are also possible.

In [10]:
    for i in range(5):
        zahnerZennium.setCurrent(i)
        time.sleep(1)
        print(f"Potential:\t{zahnerZennium.getPotential()}\tV")

Potential:	0.004607167	V
Potential:	1.913566	V
Potential:	3.952542	V
Potential:	6.068166	V
Potential:	8.130561	V


Then impedance can be measured like with the internal potentiostat.
The commands for switching the potentiostat on and off are not necessary, since the potentiostat is not used.

In [11]:
    zahnerZennium.setEISOutputPath(r"C:\THALES\temp\test1")
    zahnerZennium.setEISNaming("counter")
    zahnerZennium.setEISCounter(13)
    zahnerZennium.setEISOutputFileName("spectra")

    zahnerZennium.setAmplitude(0.5)
    zahnerZennium.setCurrent(5)
    zahnerZennium.setLowerFrequencyLimit(0.1)
    zahnerZennium.setStartFrequency(10)
    zahnerZennium.setUpperFrequencyLimit(100)
    zahnerZennium.setLowerNumberOfPeriods(3)
    zahnerZennium.setLowerStepsPerDecade(2)
    zahnerZennium.setUpperNumberOfPeriods(20)
    zahnerZennium.setUpperStepsPerDecade(3)
    zahnerZennium.setScanDirection("startToMax")
    zahnerZennium.setScanStrategy("single")
    
    zahnerZennium.measureEIS()
    
    zahnerZennium.disableFraMode()

    zenniumConnection.disconnectFromTerm()
    print("finish")

finish
